# Fundamentals of Social Data Science 2025. Week 1. Day 2. Exercises

This is a group assignment. 

You will be expected to submit an individual assignment on Tuesday at 12pm (not Monday) on Canvas. The sheet that you will be expected to submit will be released on Friday at 12pm. It is submitted on Tuesday because you will want to integrate your materials post-presentation. 

- That sheet will have a small number of individual questions related to Friday's assignment
- It will include one question about your presentation. That question is reproduced below so there are no surprises. 

The assignment submission details will be posted on Canvas under assignments.

To itemise: 
- Week 1. Day 2. Wednesday at 12pm: This "getting started" sheet is released. 
- Wednesday afternoon tutorial: We will want to ensure that you can get started on loading data. 
- Week 1. Day 3. Friday at 12pm: The individual assignment is released. 
- Week 1. Day 3. Friday afternoon tutorial: You will want to play with the Claude artifact as well as continue working with your group. 
- Week 2. Day 1. Monday at 12pm: An exercise will be released related to Network Canvas. It will require you to download Network Canvas interviewer from networkcanvas.com. 
- Week 2. Day 1. Monday afternoon tutorial: Bernie will explain the Network Canvas exercise as a part of the class. The tutorial period will be group presentations. 
- Tuesday at 12pm: Your individual assignment is due. 
- Tuesday at 12pm: Your group assignment should be posted. 

> **NOTE:** This assignment will use data from the web. This assignment has NOT been cleared for research via the CUREC process. It is an in-class assignment. Therefore, if you wish to publish anything from this analysis, you must first apply for a CUREC before publishing anything publicly with your Oxford affiliation. 

# Group exercise: Getting started

The group assignment will make use of the StackDownloader from the FSSTDS repository. This downloader (recently tested) will download, extract and process a StackExchange archive. It is pretty close to 'one click'. It creates a 'feather' archive, which is a very nice format for compressing DataFrames. You can open this in your own code. 

To begin, you will need to have everything installed for the StackDownloader. How do we do that? We install the requirements.

- **Step 1.** Clone the FSSTDS repository. 
- **Step 2.** Open the Ch.00.Stack_downloader and 'select kernel', select "Python Environments...", "Create Python Environment", "Venv -> Creates a `.venv` virtual environment in the current workspace", select **Python 3.12**. Note 3.14 is untested. Select dependencies to install -> requirements.txt. 
- **Step 3.** Run the big code cell in Stack_downloader. Select a specific archive. 
- **Step 4.** Locate and load the DataFrame. You can now use the Stack Exchange in your work. 

Note if you get errors with PyArrow below, try restarting the kernel. 

In [23]:
# In case this Jupyter Notebook is in a different repo than FSSTDS, you may need to install
# pandas and pyarrow to parse the file. 
import sys
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pandas", "pyarrow"])

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


0

## Q2. Defining helpfulness 

If you can describe the data simply then you are on your way to the big question for the group. Recall two of the trade-offs from the last lecture: "operationalisation" and "coding". The group project this week is very simple in some senses and very complex in other senses: 

Two questions: 
> - "How can we identify the most helpful users in this space" 
> - "When were the helpful users the most helpful or most active?"

So this means that your group will have to discuss:
- What defines helpfulness? Are there multiple possible metrics? 
- Do we think that a helpful person should _always_ be helpful? 
- Is helpfulness topic-specific? 
- You may want to explore wrangling the data by time. 

We do not expect you to merge in data from the users.xml / users.feather for this. However, you may want to explore how to create a datatime column. This is not covered in this lecture, but you may want to read either Chapter 10 of FSSTDS on cleaning data and Chapter 12 of FSSTDS on wrangling time data. 

You will want to divide some tasks among your group. Some might be delegated to surf the space online to come up with abductive hypotheses. Some might want to focus on rendering some charts. Some might be excellent at presentation design or at presenting to the group. Lean into your expertise and collaborate.

Presentations for this will be on Monday afternoon. The presentations will be no more than 12 minutes + 3 minutes of questions & transition. 

Each group will have a 'space' on Canvas to submit 3 things: 
- the presentation 
- the code
- the 'credits'. A single sheet (in docx/md) that details which group members participated in which ways. Treat this not merely as accountability but an opportunity to signal your own strengths. We do not expect everyone to do 1/5 of the work for every task. We do expect everyone to contribute in some way.

This code will not be graded but it will be made available to other students. 
The presentations will be given short written feedback by the instructor post-presentation.

In [24]:
# Q0. Check that you can load your own DataFrame
 
import pandas as pd 

stack_df = df = pd.read_feather("./data/vegetarianism.stackexchange.com/Posts.feather")

print(stack_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     2409 non-null   object        
 1   PostTypeId             2409 non-null   object        
 2   AcceptedAnswerId       381 non-null    object        
 3   CreationDate           2409 non-null   datetime64[ns]
 4   Score                  2409 non-null   int64         
 5   ViewCount              760 non-null    float64       
 6   Body                   2409 non-null   object        
 7   OwnerUserId            2345 non-null   object        
 8   LastEditorUserId       1418 non-null   object        
 9   LastEditDate           1438 non-null   datetime64[ns]
 10  LastActivityDate       2409 non-null   datetime64[ns]
 11  Title                  760 non-null    object        
 12  Tags                   760 non-null    object        
 13  Ans

### Load all data files

In [25]:
users = pd.read_xml("data/vegetarianism.stackexchange.com/Users.xml", xpath=".//row")
votes = pd.read_xml("data/vegetarianism.stackexchange.com/Votes.xml", xpath=".//row")
comments = pd.read_xml("data/vegetarianism.stackexchange.com/Comments.xml", xpath=".//row")
badges = pd.read_xml("data/vegetarianism.stackexchange.com/Badges.xml", xpath=".//row")
post_links = pd.read_xml("data/vegetarianism.stackexchange.com/PostLinks.xml", xpath=".//row")
post_history = pd.read_xml("data/vegetarianism.stackexchange.com/PostHistory.xml", xpath=".//row")
posts = pd.read_xml("data/vegetarianism.stackexchange.com/Posts.xml", xpath=".//row")

### Data Cleaning & Preprocessing

In [ ]:
answers = posts[posts["PostTypeId"] == 2].copy()
answers["OwnerUserId"] = answers["OwnerUserId"].fillna(-999) # replace missings with invalid id (might be deleted accounts) there are 38 of them

answers['Score'] = answers['Score'].clip(lower=0)# Assign the negative scores to 0

users['CreationDate'] = pd.to_datetime(users['CreationDate'], errors='coerce')
users['LastAccessDate'] = pd.to_datetime(users['LastAccessDate'], errors='coerce')

badges = badges[["UserId", "Name"]]
badges_by_user = badges.groupby('UserId', dropna=False)['Name'].agg(list).reset_index().rename(columns={'Name': 'badges'})

In [27]:
# compute per question total answer score
totals = answers.groupby('ParentId', dropna=False)['Score'].sum().rename('TotalAnswerScore').reset_index()

# merge question totals back onto answers
answers = answers.merge(totals, on="ParentId", how="left")

## Calculate the Score

In [28]:
import numpy as np

# Calculate normalized score
# For questions where TotalAnswerScore > 0: normalized = Score / TotalAnswerScore  
# For questions where TotalAnswerScore <= 0: we'll use the raw score (or set to 0)
answers['NormalizedScore'] = np.where(
    answers['TotalAnswerScore'] > 0,
    answers['Score'] / answers['TotalAnswerScore'],
    0  # Set to 0 for edge cases
)

print("\nNormalized score distribution:")
print(answers['NormalizedScore'].describe())
print(f"\nRange: {answers['NormalizedScore'].min():.3f} to {answers['NormalizedScore'].max():.3f}")


Normalized score distribution:
count    1497.000000
mean        0.466266
std         0.373809
min         0.000000
25%         0.125000
50%         0.375000
75%         0.969697
max         1.000000
Name: NormalizedScore, dtype: float64

Range: 0.000 to 1.000


In [29]:
# Filter out invalid users (those with OwnerUserId == -999)
valid_answers = answers[answers['OwnerUserId'] != -999].copy()
print(f"Valid answers (with known users): {len(valid_answers)}")

# Calculate user-level aggregations
user_metrics = valid_answers.groupby('OwnerUserId').agg({
    'NormalizedScore': ['mean', 'sum', 'count'],
    'Score': ['mean', 'sum'],
    'Id': 'count'  # Total number of answers per user
}).round(4)

# Flatten column names
user_metrics.columns = [
    'AvgNormalizedScore', 'TotalNormalizedScore', 'AnswerCount_norm',
    'AvgRawScore', 'TotalRawScore', 'TotalAnswers'
]

# Reset index to make OwnerUserId a column
user_metrics = user_metrics.reset_index()

print(f"\nTotal users analyzed: {len(user_metrics)}")
print(f"Users with at least 2 answers: {len(user_metrics[user_metrics['TotalAnswers'] >= 2])}")
print(f"Users with at least 5 answers: {len(user_metrics[user_metrics['TotalAnswers'] >= 5])}")

print("\nUser metrics distribution:")
print(user_metrics[['AvgNormalizedScore', 'TotalAnswers', 'AvgRawScore']].describe())


Valid answers (with known users): 1459

Total users analyzed: 406
Users with at least 2 answers: 182
Users with at least 5 answers: 62

User metrics distribution:
       AvgNormalizedScore  TotalAnswers  AvgRawScore
count          406.000000    406.000000   406.000000
mean             0.378978      3.593596     3.412230
std              0.319368      8.695954     3.137362
min              0.000000      1.000000     0.000000
25%              0.105700      1.000000     1.000000
50%              0.319450      1.000000     3.000000
75%              0.567650      3.000000     5.000000
max              1.000000    118.000000    21.750000


In [30]:
# Sort by average normalized score (descending)
top_users_by_avg = user_metrics.nlargest(10, 'AvgNormalizedScore')

print(" TOP 10 USERS BY AVERAGE NORMALIZED SCORE:")
print("=" * 70)
for idx, row in top_users_by_avg.iterrows():
    print(f"#{top_users_by_avg.index.get_loc(idx)+1:2d}. User {row['OwnerUserId']}: "
          f"Avg={row['AvgNormalizedScore']:.3f} "
          f"(Answers: {int(row['TotalAnswers'])}, "
          f"Avg Raw Score: {row['AvgRawScore']:.1f})")

# Also show top users by total contribution
top_users_by_total = user_metrics.nlargest(10, 'TotalNormalizedScore')
print("TOP 10 USERS BY TOTAL NORMALIZED CONTRIBUTION:")
print("=" * 70)
for idx, row in top_users_by_total.iterrows():
    print(f"#{top_users_by_total.index.get_loc(idx)+1:2d}. User {row['OwnerUserId']}: "
          f"Total={row['TotalNormalizedScore']:.3f} "
          f"(Answers: {int(row['TotalAnswers'])}, "
          f"Avg: {row['AvgNormalizedScore']:.3f})")


 TOP 10 USERS BY AVERAGE NORMALIZED SCORE:
# 1. User 87.0: Avg=1.000 (Answers: 1, Avg Raw Score: 8.0)
# 2. User 90.0: Avg=1.000 (Answers: 1, Avg Raw Score: 3.0)
# 3. User 186.0: Avg=1.000 (Answers: 1, Avg Raw Score: 4.0)
# 4. User 190.0: Avg=1.000 (Answers: 1, Avg Raw Score: 5.0)
# 5. User 220.0: Avg=1.000 (Answers: 1, Avg Raw Score: 2.0)
# 6. User 232.0: Avg=1.000 (Answers: 1, Avg Raw Score: 5.0)
# 7. User 317.0: Avg=1.000 (Answers: 1, Avg Raw Score: 5.0)
# 8. User 335.0: Avg=1.000 (Answers: 1, Avg Raw Score: 2.0)
# 9. User 355.0: Avg=1.000 (Answers: 1, Avg Raw Score: 2.0)
#10. User 384.0: Avg=1.000 (Answers: 1, Avg Raw Score: 4.0)
TOP 10 USERS BY TOTAL NORMALIZED CONTRIBUTION:
# 1. User 74.0: Total=72.291 (Answers: 118, Avg: 0.613)
# 2. User 1236.0: Total=59.797 (Answers: 90, Avg: 0.664)
# 3. User 164.0: Total=28.074 (Answers: 47, Avg: 0.597)
# 4. User 36.0: Total=25.223 (Answers: 43, Avg: 0.587)
# 5. User 97.0: Total=16.413 (Answers: 29, Avg: 0.566)
# 6. User 3311.0: Total=11.721 (A

In [31]:
### Normalized Score
normalized_score = answers.groupby('OwnerUserId', dropna=True).apply(lambda x: (x['Score'] / x['TotalAnswerScore']).sum()).rename('NormalizedScore').reset_index()


### Total Answer Volume
answer_volume = answers.groupby('OwnerUserId', dropna=True).size().rename('TotalAnswerVolume').reset_index()

/var/folders/6x/0dzzyfsd4078wfgbndq284380000gn/T/ipykernel_35486/1183251063.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  normalized_score = answers.groupby('OwnerUserId', dropna=True).apply(lambda x: (x['Score'] / x['TotalAnswerScore']).sum()).rename('NormalizedScore').reset_index()


In [32]:
# Create master user-level DataFrame by merging computed metrics, badges and user profile data
# Merge user_metrics (which has OwnerUserId) with normalized_score and answer_volume
master_df = user_metrics.merge(normalized_score, on='OwnerUserId', how='left')
master_df = master_df.merge(answer_volume, on='OwnerUserId', how='left')

# Merge bages to master dataframe
master_df = master_df.merge(badges_by_user, left_on='OwnerUserId', right_on="UserId", how='left')

# Merge basic user profile information from users (Id -> OwnerUserId)
users_profile = users.copy()

# Choose a subset of useful columns to bring in to avoid huge DataFrame
profile_cols = [c for c in ['Id','DisplayName','Reputation','CreationDate','LastAccessDate','Views','UpVotes','DownVotes']]
users_profile = users_profile[profile_cols]
master_df = master_df.merge(users_profile, left_on='OwnerUserId', right_on="Id", how='left')

# Set OwnerUserId as index and sort by TotalNormalizedScore desc
master_df = master_df.set_index('OwnerUserId')
master_df.drop(columns=["UserId", "Id", "NormalizedScore", "AnswerCount_norm", "TotalAnswerVolume"], inplace=True) #removing duplicate columns

# Show summary
print('Master dataframe shape:', master_df.shape)
print('Master df info:')
print(master_df.info())
print('Master df head:')
display(master_df.head())

Master dataframe shape: (406, 13)
Master df info:
<class 'pandas.core.frame.DataFrame'>
Index: 406 entries, 6.0 to 6784.0
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   AvgNormalizedScore    406 non-null    float64       
 1   TotalNormalizedScore  406 non-null    float64       
 2   AvgRawScore           406 non-null    float64       
 3   TotalRawScore         406 non-null    int64         
 4   TotalAnswers          406 non-null    int64         
 5   badges                387 non-null    object        
 6   DisplayName           406 non-null    object        
 7   Reputation            406 non-null    int64         
 8   CreationDate          406 non-null    datetime64[ns]
 9   LastAccessDate        406 non-null    datetime64[ns]
 10  Views                 406 non-null    int64         
 11  UpVotes               406 non-null    int64         
 12  DownVotes             406 no

,AvgNormalizedScore,TotalNormalizedScore,AvgRawScore,TotalRawScore,TotalAnswers,badges,DisplayName,Reputation,CreationDate,LastAccessDate,Views,UpVotes,DownVotes
OwnerUserId,,,,,,,,,,,,,
6.0,0.7276,10.1861,12.3571,173,14,"[Teacher, Student, Editor, Cleanup, Organizer,...",Riker,2420,2017-01-31 19:07:26.043,2022-08-28 21:23:28.733,131,104,17
16.0,0.1889,0.5667,4.3333,13,3,"[Teacher, Editor, Supporter, Yearling, Nice An...",mic,248,2017-01-31 19:09:02.657,2020-02-25 19:15:25.487,2,20,0
18.0,0.4667,0.4667,7.0000,7,1,"[Teacher, Student, Supporter, Autobiographer]",RocketRuwan,99,2017-01-31 19:09:30.553,2021-09-17 17:19:34.800,3,4,0
20.0,0.3193,0.9578,8.0000,24,3,"[Teacher, Student, Supporter, Critic, Yearling...",Joe,418,2017-01-31 19:09:55.527,2021-11-28 21:08:45.967,8,15,1
21.0,0.5686,2.8428,13.4000,67,5,"[Teacher, Student, Editor, Organizer, Supporte...",gerrit,1589,2017-01-31 19:09:58.223,2024-01-18 11:04:41.247,12,114,1


In [33]:
master_df.to_csv('data/master_df.csv')